In [1]:
from appgeopy import *
from my_packages import *

In [2]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


def calculate_model_metrics(
    observation: pd.Series, prediction: pd.Series
) -> dict:
    denominator = np.sum(observation)
    if denominator == 0:
        pbias = np.nan
    else:
        pbias = 100.0 * np.sum(prediction - observation) / denominator

    metrics = {
        "r_square": r2_score(observation, prediction),
        "rmse": np.sqrt(mean_squared_error(observation, prediction)),
        "mae": mean_absolute_error(observation, prediction),
        "pbias": pbias,
    }
    return metrics

['D:\\1000_SCRIPTS\\003_Project002\\20250917_GTWR002\\3D_TestRun_4\\gtwr_run_output_Layer_1\\gtwr_Layer_1_regression_kernel-tricube_lambda-0d03_bw-25_results.csv',
 'D:\\1000_SCRIPTS\\003_Project002\\20250917_GTWR002\\3D_TestRun_4\\gtwr_run_output_Layer_2\\gtwr_Layer_2_regression_kernel-tricube_lambda-0d002_bw-18_results.csv',
 'D:\\1000_SCRIPTS\\003_Project002\\20250917_GTWR002\\3D_TestRun_4\\gtwr_run_output_Layer_3\\gtwr_Layer_3_regression_kernel-tricube_lambda-0d07_bw-17_results.csv',
 'D:\\1000_SCRIPTS\\003_Project002\\20250917_GTWR002\\3D_TestRun_4\\gtwr_run_output_Layer_4\\gtwr_Layer_4_regression_kernel-tricube_lambda-0d1_bw-18_results.csv',
 'D:\\1000_SCRIPTS\\003_Project002\\20250917_GTWR002\\3D_TestRun_4\\gtwr_run_output_Layer_All\\gtwr_Layer_All_regression_kernel-tricube_lambda-0d002_bw-17_results.csv']

In [14]:
mlcw_gdf = gpd.read_file(
    r"D:\1000_SCRIPTS\003_Project002\20250222_GTWR001\2_KrigingInterpolation\points_fld\mlcw_twd97.shp"
)

found_fpath = glob(os.path.join(os.getcwd(), "*", "gtwr_Layer*.csv"))

# fpath = r".\gtwr_run_output_Layer_1\gtwr_Layer_1_regression_kernel-tricube_lambda-0d03_bw-25_results.csv"

for fpath in found_fpath:
    layer_number = os.path.basename(fpath).split("_")[2]

    dirname = os.path.dirname(fpath)
    basename = os.path.basename(fpath)

    summary_savefolder = os.path.join(dirname, "result_summary")
    summary_savefigs = os.path.join(dirname, "result_summary", "figs")

    if not os.path.exists(summary_savefigs):
        os.makedirs(summary_savefigs, exist_ok=True)

    df = pd.read_csv(fpath)
    # pointkey_arr = [
    #     f"X{int(x*1000)}Y{int(y*1000)}"
    #     for x, y in zip(df["X_TWD97"], df["Y_TWD97"])
    # ]
    # df.insert(loc=0, column="STATION", value=pointkey_arr)
    # df = df.rename({"Unnamed: 0":"id"}, axis=1)
    stations = sorted(df["input_STATION"].unique())

    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    output_df = pd.DataFrame(data=None)

    # select_station = stations[3]
    for select_station in tqdm(stations):

        corresponding_mlcw_station = mlcw_gdf.query(
            "STATION==@select_station"
        ).STATION.iloc[0]

        df_byStation = df.query("input_STATION==@select_station")

        df_byStation = df_byStation.sort_values(by="Time_value").reset_index(
            drop=True
        )

        obs_arr = df_byStation[f"input_Layer_{layer_number}"]
        sim_arr = df_byStation["predicted_value"]

        evaluation_metrics = calculate_model_metrics(obs_arr, sim_arr)
        metric_df = pd.DataFrame(
            evaluation_metrics,
            index=[corresponding_mlcw_station],
        )
        metric_df["Layer"] = f"Layer_{layer_number}"

        output_df = pd.concat([output_df, metric_df])

        fig = plt.figure(figsize=(12, 4))
        ax = fig.add_subplot(111)
        ax.plot(obs_arr, marker="o", label="Obs")
        ax.plot(sim_arr, marker="o", label="Pred")

        # Step 6: Add evaluation metrics text in lower left corner
        metrics_text = f"""R² = {evaluation_metrics['r_square']:.3f}
        RMSE = {evaluation_metrics['rmse']:.3f}
        MAE = {evaluation_metrics['mae']:.3f}
        PBIAS = {evaluation_metrics['pbias']:.1f}%"""

        # Position text at lower left: x=0.02, y=0.02 in axis coordinates
        ax.text(
            0.02,
            0.02,
            metrics_text,
            transform=ax.transAxes,  # Use axis coordinates (0-1 range)
            verticalalignment="bottom",
            bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
        )

        visualize.configure_axis(
            ax=ax,
            title=f"{corresponding_mlcw_station} - Layer {layer_number}",
            hide_spines=["top", "right"],
            tick_direction="out",
        )
        visualize.configure_legend(ax=ax)

        visualize.save_figure(
            fig=fig,
            savepath=os.path.join(
                summary_savefigs,
                f"Layer_{layer_number}_{corresponding_mlcw_station}.png",
            ),
        )

        plt.close()

    output_df.to_csv(os.path.join(summary_savefolder, basename))

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

In [11]:
corresponding_mlcw_station

'ZHENNAN'